In [1]:
import os
import sys
sys.path.insert(0, 'X:\\bayarea_urbansim\\scripts')
from baus_analytics_tools import *

In [2]:
summary_dir = 'M:\\Data\\Urban\\BAUS\\PBA50\\explore_developer_model\\runs_feasibility_all_included'
run_dir = os.path.join(summary_dir, 'runs')
trace_dir = os.path.join(summary_dir, 'trace')
parcel_trace_dir = os.path.join(summary_dir, 'trace_selected_parcels')

In [3]:
%load_ext autoreload
%autoreload 2

## helper lookups

In [4]:
# parcel to node_id/tmnode_id lookup
parcel_nodes = pd.read_csv(os.path.join(summary_dir, 'parcel_node_id.csv'))
print('{} rows of parcel to node_id/tmnode_id lookup, {} unique node_id, {} unique tmnode_id'.format(
    parcel_nodes.shape[0],
    len(parcel_nodes.node_id.unique()),
    len(parcel_nodes.tmnode_id.unique())))

print(parcel_nodes.dtypes)

1956212 rows of parcel to node_id/tmnode_id lookup, 193851 unique node_id, 11553 unique tmnode_id
parcel_id      int64
node_id        int64
tmnode_id      int64
TAZ          float64
SUPERD       float64
SD_NAME       object
dtype: object


### select certain parcels to trace

In [5]:
# parcels in SE Santa Clara county
parcels_chk_tag = 'SE_Snta_Clara'
parcels_chk = pd.read_csv(os.path.join(parcel_trace_dir, parcels_chk_tag, 'parcels_{}.csv'.format(parcels_chk_tag)))

# add nodes info
parcels_chk = parcels_chk.merge(parcel_nodes, on='parcel_id', how='left')
print(parcels_chk.shape[0])

34037


In [6]:
# # parcels in TAZ 706 (2050 HH growth NP > P)
# parcels_chk_tag = 'taz706'
# parcels_chk = pd.read_csv(os.path.join(parcel_trace_dir, parcels_chk_tag, 'parcels_{}.csv'.format(parcels_chk_tag)))

# # add nodes info
# parcels_chk = parcels_chk.merge(parcel_nodes, on='parcel_id', how='left')
# print(parcels_chk.shape[0])

In [7]:
# # parcels in TAZ 703 (2050 HH growth NP < P)
# parcels_chk_tag = 'taz703'
# parcels_chk = pd.read_csv(os.path.join(parcel_trace_dir, parcels_chk_tag, 'parcels_{}.csv'.format(parcels_chk_tag)))
# print(list(parcels_chk))

# # add nodes info
# parcels_chk = parcels_chk.merge(parcel_nodes, on='parcel_id', how='left')
# print(list(parcels_chk))
# print(parcels_chk.shape[0])

# compare selected parcels
* [parcel-level logsums](#parcel_logsum)
* [year 2010](#2010)
    * [residential hedonic models results ("rsh_simulate", "rrh_simulate")](#2010hedonic)
    * [nodes and tmnode attributes ("neighborhood_vars", "regional_vars", "price_vars")](#2010nodes)
    * [parcel_data and building_data output](#2010output)
* [year 2015-2050](#2015_2050)
    * nodes and tmnode attributes ("neighborhood_vars", "regional_vars", "price_vars")
    * alt_feasibility results
    * residential_developer model result
    * hedonic model results ("rsh_simulate", "rrh_simulate")
    * parcel_data and building_data output

## parcel-level logsums<a class="anchor" id="parcel_logsum"></a>

In [8]:
# 2010-2025
parcel_logsum_2010_2025 = pd.read_csv(os.path.join(summary_dir, 'baus_logsums_parcel_2010_2025.csv'),
                                      usecols = ['parcel_id', 'combo_logsum_np', 'combo_logsum_p', 'combo_logsum_diff'])
parcels_chk_logsum_2010_2025 = parcels_chk.merge(parcel_logsum_2010_2025, on='parcel_id', how='left')
print(parcels_chk_logsum_2010_2025.shape[0])
parcels_chk_logsum_2010_2025.to_csv(os.path.join(parcel_trace_dir, parcels_chk_tag, '{}_logsum_parcels_2010_2025.csv'.format(parcels_chk_tag)), index=False)

# 2030-2050
parcel_logsum_2030_2050 = pd.read_csv(os.path.join(summary_dir, 'baus_logsums_parcel_2030_2050.csv'),
                                      usecols = ['parcel_id', 'combo_logsum_np', 'combo_logsum_p', 'combo_logsum_diff'])
parcels_chk_logsum_2030_2050 = parcels_chk.merge(parcel_logsum_2030_2050, on='parcel_id', how='left')
print(parcels_chk_logsum_2030_2050.shape[0])
parcels_chk_logsum_2030_2050.to_csv(os.path.join(parcel_trace_dir, parcels_chk_tag, '{}_logsum_parcels_2030_2050.csv'.format(parcels_chk_tag)), index=False)


34037
34037


## year 2010 <a class="anchor" id="2010"></a>

In [9]:
year = '2010'

### residential hedonic models results ("rsh_simulate", "rrh_simulate") <a class="anchor" id="2010hedonic"></a>

In [10]:
rsh = pd.read_csv(os.path.join(trace_dir, 'rsh_simulate_by_parcel_2010.csv'))
rrh = pd.read_csv(os.path.join(trace_dir, 'rrh_simulate_by_parcel_2010.csv'))
# print(rsh.shape[0])
# print(len(rsh.parcel_id.unique()))
# print(list(rsh))
# print(list(rrh))

parcel_chk_hedonic_2010 = parcels_chk.merge(
                         rsh, on='parcel_id', how='left').merge(
                         rrh.loc[:, ~rrh.columns.isin(['unit_cnt_NP', 'unit_cnt_P', 'year'])], on='parcel_id', how='left')
print(list(parcel_chk_hedonic_2010))

# compare NP and P variables
parcel_chk_hedonic_2010_diff = \
    compare_variables_between_scenarios(parcel_chk_hedonic_2010, 
                                        'parcel_id', 'NP', 'P',
                                        compare_cols = ['unit_res_price_min', 'unit_res_price_max', 'unit_res_price_mean',
                                                        'unit_res_rent_min', 'unit_res_rent_max', 'unit_res_rent_mean',
                                                        'unit_cnt'],
                                        keep_cols = ['SUPERD', 'SD_NAME', 'TAZ', 'node_id', 'tmnode_id', '2050_hh_growth_diff'])

# add price comparison field
if 'unit_res_price_mean_P' in parcel_chk_hedonic_2010_diff:
    parcel_chk_hedonic_2010_diff.loc[:, 'unit_res_price_mean_comp'] = None
    parcel_chk_hedonic_2010_diff.loc[
        parcel_chk_hedonic_2010_diff.unit_res_price_mean_P > parcel_chk_hedonic_2010_diff.unit_res_price_mean_NP,
        'unit_res_price_mean_comp'] = 'NP < P'
    parcel_chk_hedonic_2010_diff.loc[
        parcel_chk_hedonic_2010_diff.unit_res_price_mean_P < parcel_chk_hedonic_2010_diff.unit_res_price_mean_NP,
        'unit_res_price_mean_comp'] = 'NP > P'
    parcel_chk_hedonic_2010_diff.loc[
        parcel_chk_hedonic_2010_diff.unit_res_price_mean_P == parcel_chk_hedonic_2010_diff.unit_res_price_mean_NP,
        'unit_res_price_mean_comp'] = 'NP = P'
else:
    parcel_chk_hedonic_2010_diff.loc[:, 'unit_res_price_mean_comp'] = 'NP = P'

if 'unit_res_rent_mean_P' in parcel_chk_hedonic_2010_diff:
    parcel_chk_hedonic_2010_diff.loc[:, 'unit_res_rent_mean_comp'] = None
    parcel_chk_hedonic_2010_diff.loc[
        parcel_chk_hedonic_2010_diff.unit_res_rent_mean_P > parcel_chk_hedonic_2010_diff.unit_res_rent_mean_NP,
        'unit_res_rent_mean_comp'] = 'NP < P'
    parcel_chk_hedonic_2010_diff.loc[
        parcel_chk_hedonic_2010_diff.unit_res_rent_mean_P < parcel_chk_hedonic_2010_diff.unit_res_rent_mean_NP,
        'unit_res_rent_mean_comp'] = 'NP > P'
    parcel_chk_hedonic_2010_diff.loc[
        parcel_chk_hedonic_2010_diff.unit_res_rent_mean_P == parcel_chk_hedonic_2010_diff.unit_res_rent_mean_NP,
        'unit_res_rent_mean_comp'] = 'NP = P'
else:
    parcel_chk_hedonic_2010_diff.loc[:, 'unit_res_rent_mean_comp'] = 'NP = P'

display(parcel_chk_hedonic_2010_diff)

# export
parcel_chk_hedonic_2010_diff.to_csv(os.path.join(parcel_trace_dir, 
                                                 parcels_chk_tag, 
                                                 '{}_1_parcels_hedonic_compare.csv'.format(year)), index=False)


['parcel_id', '2050_hh_growth_diff', 'node_id', 'tmnode_id', 'TAZ', 'SUPERD', 'SD_NAME', 'unit_res_price_min_NP', 'unit_res_price_max_NP', 'unit_res_price_mean_NP', 'unit_cnt_NP', 'unit_res_price_min_P', 'unit_res_price_max_P', 'unit_res_price_mean_P', 'unit_cnt_P', 'unit_res_rent_min_NP', 'unit_res_rent_max_NP', 'unit_res_rent_mean_NP', 'unit_res_rent_min_P', 'unit_res_rent_max_P', 'unit_res_rent_mean_P']
compare unit_res_price_min: not identical between NP and P
compare unit_res_price_max: not identical between NP and P
compare unit_res_price_mean: not identical between NP and P
compare unit_res_rent_min: identical between NP and P
compare unit_res_rent_max: identical between NP and P
compare unit_res_rent_mean: identical between NP and P
compare unit_cnt: identical between NP and P


C:\ProgramData\Anaconda3\envs\geo_env\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\ProgramData\Anaconda3\envs\geo_env\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,parcel_id,SUPERD,SD_NAME,TAZ,node_id,tmnode_id,2050_hh_growth_diff,unit_res_price_max_NP,unit_res_price_max_P,unit_res_price_mean_NP,unit_res_price_mean_P,unit_res_price_min_NP,unit_res_price_min_P,unit_res_price_mean_comp,unit_res_rent_mean_comp
0,1711264,14.0,SE Snta Clara Cnty,710.0,168905,5427,NP > P,280.089995,282.809139,280.089995,282.809139,280.089995,282.809139,NP < P,NP = P
1,1711387,14.0,SE Snta Clara Cnty,710.0,174035,5427,No Household,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,NP = P,NP = P
2,1711391,14.0,SE Snta Clara Cnty,710.0,155140,4151,No Household,NaN,NaN,NaN,NaN,NaN,NaN,None,NP = P
3,1717862,14.0,SE Snta Clara Cnty,705.0,303519,4151,No Household,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,NP = P,NP = P
4,1720010,14.0,SE Snta Clara Cnty,700.0,310217,10534,No Household,NaN,NaN,NaN,NaN,NaN,NaN,None,NP = P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34032,2053438,14.0,SE Snta Clara Cnty,709.0,189466,709,No Household,NaN,NaN,NaN,NaN,NaN,NaN,None,NP = P
34033,2053472,14.0,SE Snta Clara Cnty,702.0,142028,4220,NP > P,NaN,NaN,NaN,NaN,NaN,NaN,None,NP = P
34034,2053524,14.0,SE Snta Clara Cnty,698.0,186270,20436,No Household,NaN,NaN,NaN,NaN,NaN,NaN,None,NP = P
34035,2053541,14.0,SE Snta Clara Cnty,702.0,180531,11203,NP < P,NaN,NaN,NaN,NaN,NaN,NaN,None,NP = P


### nodes and tmnode attributes ("neighborhood_vars", "regional_vars", "price_vars")<a class="anchor" id="2010nodes"></a>

In [11]:
nodes_attr = pd.read_csv(os.path.join(trace_dir, 'price_vars_2010.csv'))
# print(list(nodes_attr))
tmnodes_attr = pd.read_csv(os.path.join(trace_dir, 'regional_vars_2010.csv'))
# print('\n', list(tmnodes_attr))

parcel_chk_nodes_2010 = parcels_chk.merge(
    nodes_attr, on='node_id', how='left').merge(
    tmnodes_attr, on='tmnode_id', how='left')
print(parcel_chk_nodes_2010.shape[0])


# compare NP and P variables
parcel_chk_nodes_2010_diff = \
    compare_variables_between_scenarios(parcel_chk_nodes_2010, 
                                        'parcel_id', 'NP', 'P',
                                        compare_cols = ['residential_units_45', 'jobs_15', 'jobs_45',
                                                        'sum_income_40', 'embarcadero', 'pacheights', 'stanford',
                                                        'retail_sqft_3000', 'sum_income_3000', 
                                                        'residential_units_500', 'residential_units_1500', 
                                                        'office_1500', 'retail_1500', 'industrial_1500', 
                                                        'ave_sqft_per_unit', 'ave_lot_size_per_unit', 
                                                        'population', 'poor', 'renters', 'sfdu', 'ave_hhsize', 
                                                        'jobs_500', 'jobs_1500', 'ave_income_1500', 'ave_income_500', 
                                                        'retail_ratio', 'residential', 'retail', 'office', 'industrial'],
                                       keep_cols = ['SUPERD', 'SD_NAME', 'TAZ', 'node_id', 'tmnode_id', '2050_hh_growth_diff'])

# compare 'residential' column
if 'residential_P' in parcel_chk_nodes_2010_diff:
    parcel_chk_nodes_2010_diff.loc[:, 'residential_price_comp'] = None
    parcel_chk_nodes_2010_diff.loc[
        parcel_chk_nodes_2010_diff.residential_P > parcel_chk_nodes_2010_diff.residential_NP,
        'residential_price_comp'] = 'NP < P'
    parcel_chk_nodes_2010_diff.loc[
        parcel_chk_nodes_2010_diff.residential_P < parcel_chk_nodes_2010_diff.residential_NP,
        'residential_price_comp'] = 'NP > P'
    parcel_chk_nodes_2010_diff.loc[
        parcel_chk_nodes_2010_diff.residential_P == parcel_chk_nodes_2010_diff.residential_NP,
        'residential_price_comp'] = 'NP = P'

    display(parcel_chk_nodes_2010_diff)

# export
parcel_chk_nodes_2010_diff.to_csv(os.path.join(parcel_trace_dir,
                                               parcels_chk_tag, 
                                               '{}_2_parcels_nodes_compare.csv'.format(year)), index=False)


34037
compare residential_units_45: identical between NP and P
compare jobs_15: identical between NP and P
compare jobs_45: identical between NP and P
compare sum_income_40: identical between NP and P
compare embarcadero: identical between NP and P
compare pacheights: identical between NP and P
compare stanford: identical between NP and P
compare retail_sqft_3000: identical between NP and P
compare sum_income_3000: identical between NP and P
compare residential_units_500: identical between NP and P
compare residential_units_1500: identical between NP and P
compare office_1500: identical between NP and P
compare retail_1500: identical between NP and P
compare industrial_1500: identical between NP and P
compare ave_sqft_per_unit: identical between NP and P
compare ave_lot_size_per_unit: identical between NP and P
compare population: identical between NP and P
compare poor: identical between NP and P
compare renters: identical between NP and P
compare sfdu: identical between NP and P
comp

C:\ProgramData\Anaconda3\envs\geo_env\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\ProgramData\Anaconda3\envs\geo_env\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,parcel_id,SUPERD,SD_NAME,TAZ,node_id,tmnode_id,2050_hh_growth_diff,residential_NP,residential_P,residential_price_comp
0,1711264,14.0,SE Snta Clara Cnty,710.0,168905,5427,NP > P,508.675812,508.675812,NP = P
1,1711387,14.0,SE Snta Clara Cnty,710.0,174035,5427,No Household,507.185089,507.185089,NP = P
2,1711391,14.0,SE Snta Clara Cnty,710.0,155140,4151,No Household,508.500885,508.500885,NP = P
3,1717862,14.0,SE Snta Clara Cnty,705.0,303519,4151,No Household,250.000000,250.000000,NP = P
4,1720010,14.0,SE Snta Clara Cnty,700.0,310217,10534,No Household,574.519043,574.519043,NP = P
...,...,...,...,...,...,...,...,...,...,...
34032,2053438,14.0,SE Snta Clara Cnty,709.0,189466,709,No Household,527.276367,527.276367,NP = P
34033,2053472,14.0,SE Snta Clara Cnty,702.0,142028,4220,NP > P,612.545166,612.545166,NP = P
34034,2053524,14.0,SE Snta Clara Cnty,698.0,186270,20436,No Household,420.584869,421.027893,NP < P
34035,2053541,14.0,SE Snta Clara Cnty,702.0,180531,11203,NP < P,607.617798,607.617798,NP = P


### parcel_data and building_data output<a class="anchor" id="2010output"></a>

In [12]:
parcel_data = pd.read_csv(os.path.join(trace_dir, 'parcel_data_compare_2010.csv'))
blg_data = pd.read_csv(os.path.join(trace_dir, 'building_data_parcel_compare_2010.csv'))

parcels_chk_final_2010 = parcels_chk.merge(
                         parcel_data, on='parcel_id', how='left').merge(
                         blg_data, on='parcel_id', how='left')

# fillin na
for i in ['zoned_du', 'zoned_du_underbuild',
          'residential_units', 'resUnits_nondev', 'tothh']:
    parcels_chk_final_2010[i+'_NP'].fillna(0, inplace=True)
    parcels_chk_final_2010[i+'_P'].fillna(0, inplace=True)

# compare NP and P
parcels_chk_final_2010_diff = \
    compare_variables_between_scenarios(parcels_chk_final_2010, 'parcel_id', 'NP', 'P',
                                        compare_cols = ['zoned_du', 'zoned_du_underbuild', 
                                                        'residential_units', 'resUnits_nondev', 'unit_price_mean_nondev',
                                                        'tothh'],
                                        keep_cols = ['SUPERD', 'SD_NAME', 'TAZ', 'node_id', 'tmnode_id', '2050_hh_growth_diff'])

# add back key fields for analysis
parcels_chk_final_2010_diff = parcels_chk_final_2010_diff.merge(
                              parcels_chk_final_2010[['parcel_id', 'residential_units_NP', 'residential_units_P',
                                                      'resUnits_nondev_NP', 'resUnits_nondev_P', 'tothh_NP', 'tothh_P']])

# add HH and units comparison column
parcels_chk_final_2010_diff.loc[:, 'tothh_comp'] = None
parcels_chk_final_2010_diff.loc[
    parcels_chk_final_2010_diff.tothh_P > parcels_chk_final_2010_diff.tothh_NP,
    'tothh_comp'] = 'NP < P'
parcels_chk_final_2010_diff.loc[
    parcels_chk_final_2010_diff.tothh_P < parcels_chk_final_2010_diff.tothh_NP,
    'tothh_comp'] = 'NP > P'
parcels_chk_final_2010_diff.loc[
    parcels_chk_final_2010_diff.tothh_P == parcels_chk_final_2010_diff.tothh_NP,
    'tothh_comp'] = 'NP = P'
parcels_chk_final_2010_diff.loc[
    (parcels_chk_final_2010_diff.tothh_P == 0) & (parcels_chk_final_2010_diff.tothh_NP == 0),
    'tothh_comp'] = 'No Household'

parcels_chk_final_2010_diff.loc[:, 'totUnits_comp'] = None
parcels_chk_final_2010_diff.loc[
    parcels_chk_final_2010_diff.residential_units_P > parcels_chk_final_2010_diff.residential_units_NP,
    'totUnits_comp'] = 'NP < P'
parcels_chk_final_2010_diff.loc[
    parcels_chk_final_2010_diff.residential_units_P < parcels_chk_final_2010_diff.residential_units_NP,
    'totUnits_comp'] = 'NP > P'
parcels_chk_final_2010_diff.loc[
    parcels_chk_final_2010_diff.residential_units_P == parcels_chk_final_2010_diff.residential_units_NP,
    'totUnits_comp'] = 'NP = P'
parcels_chk_final_2010_diff.loc[
    (parcels_chk_final_2010_diff.residential_units_P == 0) & (parcels_chk_final_2010_diff.residential_units_NP == 0),
    'totUnits_comp'] = 'No Housing'

display(parcels_chk_final_2010_diff)

# export
parcels_chk_final_2010_diff.to_csv(os.path.join(parcel_trace_dir,
                                                parcels_chk_tag, 
                                                '{}_3_parcels_final_compare.csv'.format(year)), index=False)


compare zoned_du: identical between NP and P
compare zoned_du_underbuild: identical between NP and P
compare residential_units: identical between NP and P
compare resUnits_nondev: identical between NP and P
compare unit_price_mean_nondev: not identical between NP and P
compare tothh: not identical between NP and P


,parcel_id,SUPERD,SD_NAME,TAZ,node_id,tmnode_id,2050_hh_growth_diff,tothh_NP,tothh_P,unit_price_mean_nondev_NP,unit_price_mean_nondev_P,residential_units_NP,residential_units_P,resUnits_nondev_NP,resUnits_nondev_P,tothh_comp,totUnits_comp
0,1711264,14.0,SE Snta Clara Cnty,710.0,168905,5427,NP > P,0.0,1.0,780827.952851,780827.952851,1.0,1.0,1.0,1.0,NP < P,NP = P
1,1711387,14.0,SE Snta Clara Cnty,710.0,174035,5427,No Household,1.0,1.0,752126.202689,752126.202689,1.0,1.0,1.0,1.0,NP = P,NP = P
2,1711391,14.0,SE Snta Clara Cnty,710.0,155140,4151,No Household,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,No Household,No Housing
3,1717862,14.0,SE Snta Clara Cnty,705.0,303519,4151,No Household,1.0,1.0,683140.077533,683140.077533,1.0,1.0,1.0,1.0,NP = P,NP = P
4,1720010,14.0,SE Snta Clara Cnty,700.0,310217,10534,No Household,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,No Household,No Housing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34032,2053438,14.0,SE Snta Clara Cnty,709.0,189466,709,No Household,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,No Household,No Housing
34033,2053472,14.0,SE Snta Clara Cnty,702.0,142028,4220,NP > P,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,No Household,No Housing
34034,2053524,14.0,SE Snta Clara Cnty,698.0,186270,20436,No Household,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,No Household,No Housing
34035,2053541,14.0,SE Snta Clara Cnty,702.0,180531,11203,NP < P,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,No Household,No Housing


## year 2015-2050 <a class="anchor" id="2015_2050"></a>

In [13]:
for year_num in range(2015, 2055, 5):
    year = str(year_num)
    print('################################ ', year)

    #################### nodes and tmnode attributes ("neighborhood_vars", "regional_vars", "price_vars")
    print('############ nodes and tmnode attributes ("neighborhood_vars", "regional_vars", "price_vars")')
    nodes_attr = pd.read_csv(os.path.join(trace_dir, 'price_vars_{}.csv'.format(year)))
    # print(list(nodes_attr))
    tmnodes_attr = pd.read_csv(os.path.join(trace_dir, 'regional_vars_{}.csv'.format(year)))
    # print('\n', list(tmnodes_attr))

    parcels_chk_nodes = parcels_chk.merge(
        nodes_attr, on='node_id', how='left').merge(
        tmnodes_attr, on='tmnode_id', how='left')
    print(parcels_chk_nodes.shape[0])

    # compare attributes
    parcels_chk_nodes_diff = \
        compare_variables_between_scenarios(parcels_chk_nodes, 'parcel_id', 'NP', 'P',
                                            compare_cols = ['retail_sqft_3000', 'sum_income_3000', 
                                                            'residential_units_500', 'residential_units_1500', 
                                                            'office_1500', 'retail_1500', 'industrial_1500', 
                                                            'ave_sqft_per_unit', 'ave_lot_size_per_unit', 
                                                            'population', 'poor', 'renters', 'sfdu', 'ave_hhsize', 
                                                            'jobs_500', 'jobs_1500', 'ave_income_1500', 'ave_income_500', 
                                                            'retail_ratio', 'residential', 'retail', 'office', 'industrial', 
                                                            'residential_units_45', 'jobs_15', 'jobs_45',
                                                            'sum_income_40', 'embarcadero', 'pacheights', 'stanford'],
                                            keep_cols = ['SUPERD', 'SD_NAME', 'TAZ', 'node_id', 'tmnode_id', '2050_hh_growth_diff'])
    
    # compare 'residential' column
    if 'residential_P' in parcels_chk_nodes_diff:
        parcels_chk_nodes_diff.loc[:, 'residential_price_comp'] = None
        parcels_chk_nodes_diff.loc[
            parcels_chk_nodes_diff.residential_P > parcels_chk_nodes_diff.residential_NP,
            'residential_price_comp'] = 'NP < P'
        parcels_chk_nodes_diff.loc[
            parcels_chk_nodes_diff.residential_P < parcels_chk_nodes_diff.residential_NP,
            'residential_price_comp'] = 'NP > P'
        parcels_chk_nodes_diff.loc[
            parcels_chk_nodes_diff.residential_P == parcels_chk_nodes_diff.residential_NP,
            'residential_price_comp'] = 'NP = P'
    else:
        parcels_chk_nodes_diff.loc[:, 'residential_price_comp'] = 'NP = P'
    
    display(parcels_chk_nodes_diff)

    # export
    parcels_chk_nodes_diff.to_csv(os.path.join(parcel_trace_dir,
                                               parcels_chk_tag, 
                                               '{}_1_parcels_nodes_compare.csv'.format(year)), index=False)


    #################### alt_feasibility results
    print('############ alt_feasibility results')

    feasibility_compare = pd.read_csv(os.path.join(trace_dir, 'feasibility_compare_{}.csv'.format(year)))
    print(list(feasibility_compare))

    parcel_feasibility = parcels_chk.merge(feasibility_compare, on='parcel_id', how='left')
    # parcel_feasibility.sort_index(axis=1, inplace=True)

    print(parcel_feasibility.max_profit_chk.value_counts())

    # compare NP and P
    parcel_feasibility_diff = \
        compare_variables_between_scenarios(parcel_feasibility, 'parcel_id', 'NP', 'P',
                                            compare_cols = ['total_cost', 'building_revenue', 
                                                            'max_profit_far', 'max_profit',
                                                            'residential_sales_price_sqft', 'residential_sqft'],
                                            keep_cols = ['SUPERD', 'SD_NAME', 'TAZ', 'node_id', 'tmnode_id', '2050_hh_growth_diff'])
    # add back "max_profit_chk" columns
    parcel_feasibility_diff = parcel_feasibility_diff.merge(
                                   parcel_feasibility[['parcel_id', 'max_profit_chk']], on='parcel_id', how='left')
    display(parcel_feasibility_diff)

    # export
    parcel_feasibility_diff.to_csv(os.path.join(parcel_trace_dir,
                                                parcels_chk_tag, 
                                                '{}_2_parcels_feasibility_compare.csv'.format(year)), index=False)



    #################### residential_developer model result
    print('############ residential_developer model result')
    resDev_compare = pd.read_csv(os.path.join(trace_dir, 'resDevModel_by_parcel_{}.csv'.format(year)))
    print(list(resDev_compare))

    parcel_chk_resDev = parcels_chk.merge(resDev_compare, on='parcel_id', how='left')
    # print(parcel_resDev.shape[0])

    # add residential_units_compare field
    parcel_chk_resDev['residential_units_NP'].fillna(0, inplace=True)
    parcel_chk_resDev['residential_units_P'].fillna(0, inplace=True)
    
    parcel_chk_resDev.loc[:, 'residential_units_comp'] = None
    parcel_chk_resDev.loc[
        parcel_chk_resDev.residential_units_NP < parcel_chk_resDev.residential_units_P,
        'residential_units_comp'] = 'NP < P'
    parcel_chk_resDev.loc[
        parcel_chk_resDev.residential_units_NP > parcel_chk_resDev.residential_units_P,
        'residential_units_comp'] = 'NP > P'
    parcel_chk_resDev.loc[
        parcel_chk_resDev.residential_units_NP == parcel_chk_resDev.residential_units_P,
        'residential_units_comp'] = 'NP = P'
    parcel_chk_resDev.loc[
        (parcel_chk_resDev.residential_units_NP == 0) & (parcel_chk_resDev.residential_units_P == 0),
        'residential_units_comp'] = 'no development'
    
    display(parcel_chk_resDev)

    # export
    parcel_chk_resDev.to_csv(os.path.join(parcel_trace_dir,
                                          parcels_chk_tag, 
                                          '{}_3_parcels_resDev_compare.csv'.format(year)), index=False)



    #################### hedonic model results ("rsh_simulate", "rrh_simulate")
    print('############ hedonic model results ("rsh_simulate", "rrh_simulate")')
    rsh = pd.read_csv(os.path.join(trace_dir, 'rsh_simulate_by_parcel_{}.csv'.format(year)))
    rrh = pd.read_csv(os.path.join(trace_dir, 'rrh_simulate_by_parcel_{}.csv'.format(year)))

    parcel_chk_hedonic = parcels_chk.merge(
                              rsh, on='parcel_id', how='left').merge(
                              rrh.loc[:, ~rrh.columns.isin(['unit_cnt_NP', 'unit_cnt_P', 'year'])], on='parcel_id', how='left')

    # compare NP and P variables
    parcel_chk_hedonic_diff = \
        compare_variables_between_scenarios(parcel_chk_hedonic, 
                                            'parcel_id', 'NP', 'P',
                                             compare_cols = ['unit_res_price_min', 'unit_res_price_max', 'unit_res_price_mean',
                                                            'unit_res_rent_min', 'unit_res_rent_max', 'unit_res_rent_mean',
                                                            'unit_cnt'],
                                             keep_cols = ['SUPERD', 'SD_NAME', 'TAZ', 'node_id', 'tmnode_id', '2050_hh_growth_diff'])

    # add price comparison field
    if 'unit_res_price_mean_P' in parcel_chk_hedonic_diff:
        parcel_chk_hedonic_diff.loc[:, 'unit_res_price_mean_comp'] = None
        parcel_chk_hedonic_diff.loc[
            parcel_chk_hedonic_diff.unit_res_price_mean_P > parcel_chk_hedonic_diff.unit_res_price_mean_NP,
            'unit_res_price_mean_comp'] = 'NP < P'
        parcel_chk_hedonic_diff.loc[
            parcel_chk_hedonic_diff.unit_res_price_mean_P < parcel_chk_hedonic_diff.unit_res_price_mean_NP,
            'unit_res_price_mean_comp'] = 'NP > P'
        parcel_chk_hedonic_diff.loc[
            parcel_chk_hedonic_diff.unit_res_price_mean_P == parcel_chk_hedonic_diff.unit_res_price_mean_NP,
            'unit_res_price_mean_comp'] = 'NP = P'
    else:
        parcel_chk_hedonic_diff.loc[:, 'unit_res_price_mean_comp'] = 'NP = P'

    if 'unit_res_rent_mean_P' in parcel_chk_hedonic_diff:
        parcel_chk_hedonic_diff.loc[:, 'unit_res_rent_mean_comp'] = None
        parcel_chk_hedonic_diff.loc[
            parcel_chk_hedonic_diff.unit_res_rent_mean_P > parcel_chk_hedonic_diff.unit_res_rent_mean_NP,
            'unit_res_rent_mean_comp'] = 'NP < P'
        parcel_chk_hedonic_diff.loc[
            parcel_chk_hedonic_diff.unit_res_rent_mean_P < parcel_chk_hedonic_diff.unit_res_rent_mean_NP,
            'unit_res_rent_mean_comp'] = 'NP > P'
        parcel_chk_hedonic_diff.loc[
            parcel_chk_hedonic_diff.unit_res_rent_mean_P == parcel_chk_hedonic_diff.unit_res_rent_mean_NP,
            'unit_res_rent_mean_comp'] = 'NP = P'
    else:
        parcel_chk_hedonic_diff.loc[:, 'unit_res_rent_mean_comp'] = 'NP = P'
    
    display(parcel_chk_hedonic_diff)

    # export
    parcel_chk_hedonic_diff.to_csv(os.path.join(parcel_trace_dir,
                                                parcels_chk_tag,
                                                '{}_4_parcels_hedonic_compare.csv'.format(year)), index=False)


    #################### parcel_data and building_data output
    print('############ parcel_data and building_data output')
    parcel_data = pd.read_csv(os.path.join(trace_dir, 'parcel_data_compare_{}.csv'.format(year)))
    blg_data = pd.read_csv(os.path.join(trace_dir, 'building_data_parcel_compare_{}.csv'.format(year)))

    parcels_chk_final = parcels_chk.merge(
                             parcel_data, on='parcel_id', how='left').merge(
                             blg_data, on='parcel_id', how='left')
    
    # fillin na
    for i in ['zoned_du', 'zoned_du_underbuild',
              'residential_units', 'resUnits_nondev', 'resUnits_dev', 'tothh']:
        parcels_chk_final[i+'_NP'].fillna(0, inplace=True)
        parcels_chk_final[i+'_P'].fillna(0, inplace=True)
    
    # compare NP and P
    parcels_chk_final_diff = \
        compare_variables_between_scenarios(parcels_chk_final, 'parcel_id', 'NP', 'P',
                                            compare_cols = ['zoned_du', 'zoned_du_underbuild', 
                                                            'residential_units',
                                                            'resUnits_nondev', 'unit_price_mean_nondev',
                                                            'resUnits_dev', 'unit_price_mean_dev',
                                                            'tothh'],
                                            keep_cols = ['SUPERD', 'SD_NAME', 'TAZ', 'node_id', 'tmnode_id',
                                                         '2050_hh_growth_diff'])
    # add back key fields for analysis
    parcels_chk_final_diff = parcels_chk_final_diff.merge(
                             parcels_chk_final[['parcel_id', 'residential_units_NP', 'residential_units_P',
                                                'resUnits_nondev_NP', 'resUnits_nondev_P',
                                                'resUnits_dev_NP', 'resUnits_dev_P', 'tothh_NP', 'tothh_P']])
    # add HH and units comparison column
    parcels_chk_final_diff.loc[:, 'tothh_comp'] = None
    parcels_chk_final_diff.loc[
        parcels_chk_final_diff.tothh_P > parcels_chk_final_diff.tothh_NP,
        'tothh_comp'] = 'NP < P'
    parcels_chk_final_diff.loc[
        parcels_chk_final_diff.tothh_P < parcels_chk_final_diff.tothh_NP,
        'tothh_comp'] = 'NP > P'
    parcels_chk_final_diff.loc[
        parcels_chk_final_diff.tothh_P == parcels_chk_final_diff.tothh_NP,
        'tothh_comp'] = 'NP = P'
    parcels_chk_final_diff.loc[
        (parcels_chk_final_diff.tothh_P == 0) & (parcels_chk_final_diff.tothh_NP == 0),
        'tothh_comp'] = 'No Household'
    
    parcels_chk_final_diff.loc[:, 'totUnits_comp'] = None
    parcels_chk_final_diff.loc[
        parcels_chk_final_diff.residential_units_P > parcels_chk_final_diff.residential_units_NP,
        'totUnits_comp'] = 'NP < P'
    parcels_chk_final_diff.loc[
        parcels_chk_final_diff.residential_units_P < parcels_chk_final_diff.residential_units_NP,
        'totUnits_comp'] = 'NP > P'
    parcels_chk_final_diff.loc[
        parcels_chk_final_diff.residential_units_P == parcels_chk_final_diff.residential_units_NP,
        'totUnits_comp'] = 'NP = P'
    parcels_chk_final_diff.loc[
        (parcels_chk_final_diff.residential_units_P == 0) & (parcels_chk_final_diff.residential_units_NP == 0),
        'totUnits_comp'] = 'No Housing'
    
    display(parcels_chk_final_diff)

    # export
    parcels_chk_final_diff.to_csv(os.path.join(parcel_trace_dir,
                                               parcels_chk_tag,
                                               '{}_5_parcels_final_compare.csv'.format(year)), index=False)


################################  2015
############ nodes and tmnode attributes ("neighborhood_vars", "regional_vars", "price_vars")
34037
compare retail_sqft_3000: identical between NP and P
compare sum_income_3000: identical between NP and P
compare residential_units_500: identical between NP and P
compare residential_units_1500: identical between NP and P
compare office_1500: identical between NP and P
compare retail_1500: identical between NP and P
compare industrial_1500: identical between NP and P
compare ave_sqft_per_unit: identical between NP and P
compare ave_lot_size_per_unit: identical between NP and P
compare population: identical between NP and P
compare poor: identical between NP and P
compare renters: identical between NP and P
compare sfdu: identical between NP and P
compare ave_hhsize: identical between NP and P
compare jobs_500: not identical between NP and P
compare jobs_1500: not identical between NP and P
compare ave_income_1500: identical between NP and P
compare 

C:\ProgramData\Anaconda3\envs\geo_env\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\ProgramData\Anaconda3\envs\geo_env\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,parcel_id,SUPERD,SD_NAME,TAZ,node_id,tmnode_id,2050_hh_growth_diff,industrial_NP,industrial_P,jobs_1500_NP,...,jobs_15_P,jobs_45_NP,jobs_45_P,jobs_500_NP,jobs_500_P,office_NP,office_P,residential_NP,residential_P,residential_price_comp
0,1711264,14.0,SE Snta Clara Cnty,710.0,168905,5427,NP > P,7.388745,7.364560,6.534995,...,9.789681,12.155181,12.155004,4.797022,4.799449,13.857484,13.820658,353.453186,353.536041,NP < P
1,1711387,14.0,SE Snta Clara Cnty,710.0,174035,5427,No Household,7.388745,7.364560,6.590234,...,9.789681,12.155181,12.155004,3.666249,3.673632,13.857484,13.820658,352.699310,352.936676,NP < P
2,1711391,14.0,SE Snta Clara Cnty,710.0,155140,4151,No Household,7.393150,7.368951,6.271308,...,9.752893,12.105490,12.105308,5.426705,5.427999,13.857484,13.820658,353.211365,353.294159,NP < P
3,1717862,14.0,SE Snta Clara Cnty,705.0,303519,4151,No Household,-1.000000,-1.000000,4.057982,...,9.752893,12.105490,12.105308,0.000000,0.000000,10.804435,10.775215,200.000000,200.000000,NP = P
4,1720010,14.0,SE Snta Clara Cnty,700.0,310217,10534,No Household,7.855904,7.855992,6.996161,...,10.195743,12.819112,12.819048,6.217203,6.217203,11.677463,11.677657,402.320648,402.414948,NP < P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34032,2053438,14.0,SE Snta Clara Cnty,709.0,189466,709,No Household,7.108380,7.085003,6.535657,...,9.850502,12.242480,12.242309,4.998716,5.010146,13.087624,13.052739,366.253052,366.338898,NP < P
34033,2053472,14.0,SE Snta Clara Cnty,702.0,142028,4220,NP > P,7.913579,7.913662,5.956763,...,10.109794,12.759314,12.759241,3.740555,3.740555,12.011006,12.011198,426.187958,426.287842,NP < P
34034,2053524,14.0,SE Snta Clara Cnty,698.0,186270,20436,No Household,7.158350,7.113240,4.078222,...,10.135735,12.969353,12.969305,1.526843,1.526843,14.859439,14.859622,370.570038,371.963196,NP < P
34035,2053541,14.0,SE Snta Clara Cnty,702.0,180531,11203,NP < P,8.238580,8.238649,5.395376,...,9.867036,12.797729,12.797672,0.000000,0.000000,13.534012,13.534211,426.173462,426.273346,NP < P


############ alt_feasibility results
['parcel_id', 'superdistrict_NP', 'total_cost_NP', 'building_revenue_NP', 'max_profit_far_NP', 'max_profit_NP', 'residential_sales_price_sqft_NP', 'residential_sqft_NP', 'superdistrict_P', 'total_cost_P', 'building_revenue_P', 'max_profit_far_P', 'max_profit_P', 'residential_sales_price_sqft_P', 'residential_sqft_P', 'max_profit_chk']
Both profitable        3471
Both not profitable    2577
not allow                35
P profitable only         7
Name: max_profit_chk, dtype: int64
compare total_cost: not identical between NP and P
compare building_revenue: not identical between NP and P
compare max_profit_far: not identical between NP and P
compare max_profit: not identical between NP and P
compare residential_sales_price_sqft: not identical between NP and P
compare residential_sqft: not identical between NP and P


,parcel_id,SUPERD,SD_NAME,TAZ,node_id,tmnode_id,2050_hh_growth_diff,building_revenue_NP,building_revenue_P,max_profit_NP,max_profit_P,max_profit_far_NP,max_profit_far_P,residential_sales_price_sqft_NP,residential_sales_price_sqft_P,residential_sqft_NP,residential_sqft_P,total_cost_NP,total_cost_P,max_profit_chk
0,1711264,14.0,SE Snta Clara Cnty,710.0,168905,5427,NP > P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1711387,14.0,SE Snta Clara Cnty,710.0,174035,5427,No Household,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1711391,14.0,SE Snta Clara Cnty,710.0,155140,4151,No Household,1.377970e+07,1.378293e+07,1.786888e+06,1.793182e+06,2.75,2.75,353.211365,353.294159,48765.764319,48765.764319,1.199281e+07,1.198975e+07,Both profitable
3,1717862,14.0,SE Snta Clara Cnty,705.0,303519,4151,No Household,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1720010,14.0,SE Snta Clara Cnty,700.0,310217,10534,No Household,1.446759e+06,1.447098e+06,-3.903412e+05,-3.901510e+05,2.00,2.00,402.320648,402.414948,3596.034916,3596.034916,1.837100e+06,1.837249e+06,Both not profitable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34032,2053438,14.0,SE Snta Clara Cnty,709.0,189466,709,No Household,4.935879e+07,4.937035e+07,1.329740e+07,1.330865e+07,1.50,1.50,366.253052,366.338898,134766.892813,134766.892813,3.606139e+07,3.606170e+07,Both profitable
34033,2053472,14.0,SE Snta Clara Cnty,702.0,142028,4220,NP > P,8.351885e+06,8.353842e+06,3.228304e+06,3.230261e+06,1.50,1.50,426.187958,426.287842,19596.717642,19596.717642,5.123581e+06,5.123581e+06,Both profitable
34034,2053524,14.0,SE Snta Clara Cnty,698.0,186270,20436,No Household,1.792053e+06,1.798791e+06,4.072686e+05,4.140058e+05,2.00,2.00,370.570038,371.963196,4835.937310,4835.937310,1.384785e+06,1.384785e+06,Both profitable
34035,2053541,14.0,SE Snta Clara Cnty,702.0,180531,11203,NP < P,7.658838e+07,7.660633e+07,2.820489e+07,2.822235e+07,1.50,1.50,426.173462,426.273346,179711.743206,179711.743206,4.838349e+07,4.838398e+07,Both profitable


...

############ hedonic model results ("rsh_simulate", "rrh_simulate")
compare unit_res_price_min: not identical between NP and P
compare unit_res_price_max: not identical between NP and P
compare unit_res_price_mean: not identical between NP and P
compare unit_res_rent_min: not identical between NP and P
compare unit_res_rent_max: not identical between NP and P
compare unit_res_rent_mean: not identical between NP and P
compare unit_cnt: not identical between NP and P


C:\ProgramData\Anaconda3\envs\geo_env\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\ProgramData\Anaconda3\envs\geo_env\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,parcel_id,SUPERD,SD_NAME,TAZ,node_id,tmnode_id,2050_hh_growth_diff,unit_cnt_NP,unit_cnt_P,unit_res_price_max_NP,...,unit_res_price_min_NP,unit_res_price_min_P,unit_res_rent_max_NP,unit_res_rent_max_P,unit_res_rent_mean_NP,unit_res_rent_mean_P,unit_res_rent_min_NP,unit_res_rent_min_P,unit_res_price_mean_comp,unit_res_rent_mean_comp
0,1711264,14.0,SE Snta Clara Cnty,710.0,168905,5427,NP > P,1.0,1.0,301.659384,...,301.659384,312.787906,1.930008,1.938838,1.930008,1.938838,1.930008,1.938838,NP < P,NP < P
1,1711387,14.0,SE Snta Clara Cnty,710.0,174035,5427,No Household,1.0,1.0,200.000000,...,200.000000,200.000000,1.960050,1.972995,1.960050,1.972995,1.960050,1.972995,NP = P,NP < P
2,1711391,14.0,SE Snta Clara Cnty,710.0,155140,4151,No Household,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
3,1717862,14.0,SE Snta Clara Cnty,705.0,303519,4151,No Household,1.0,1.0,200.000000,...,200.000000,200.000000,1.088507,1.087579,1.088507,1.087579,1.088507,1.087579,NP = P,NP > P
4,1720010,14.0,SE Snta Clara Cnty,700.0,310217,10534,No Household,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34032,2053438,14.0,SE Snta Clara Cnty,709.0,189466,709,No Household,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
34033,2053472,14.0,SE Snta Clara Cnty,702.0,142028,4220,NP > P,20.0,NaN,419.899994,...,419.899994,NaN,2.413682,NaN,2.413682,NaN,2.413682,NaN,None,None
34034,2053524,14.0,SE Snta Clara Cnty,698.0,186270,20436,No Household,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
34035,2053541,14.0,SE Snta Clara Cnty,702.0,180531,11203,NP < P,NaN,180.0,NaN,...,NaN,200.000000,NaN,2.519981,NaN,2.519981,NaN,2.519981,None,None


############ parcel_data and building_data output
compare zoned_du: identical between NP and P
compare zoned_du_underbuild: not identical between NP and P
compare residential_units: not identical between NP and P
compare resUnits_nondev: not identical between NP and P
compare unit_price_mean_nondev: not identical between NP and P
compare resUnits_dev: not identical between NP and P
compare unit_price_mean_dev: not identical between NP and P
compare tothh: not identical between NP and P


,parcel_id,SUPERD,SD_NAME,TAZ,node_id,tmnode_id,2050_hh_growth_diff,resUnits_dev_NP,resUnits_dev_P,resUnits_nondev_NP,...,tothh_NP,tothh_P,unit_price_mean_dev_NP,unit_price_mean_dev_P,unit_price_mean_nondev_NP,unit_price_mean_nondev_P,zoned_du_underbuild_NP,zoned_du_underbuild_P,tothh_comp,totUnits_comp
0,1711264,14.0,SE Snta Clara Cnty,710.0,168905,5427,NP > P,0.0,0.0,1.0,...,1.0,1.0,NaN,NaN,551763.379037,559877.832074,5,5,NP = P,NP = P
1,1711387,14.0,SE Snta Clara Cnty,710.0,174035,5427,No Household,0.0,0.0,1.0,...,1.0,1.0,NaN,NaN,530468.683673,539041.796642,4,4,NP = P,NP = P
2,1711391,14.0,SE Snta Clara Cnty,710.0,155140,4151,No Household,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,52,52,No Household,No Housing
3,1717862,14.0,SE Snta Clara Cnty,705.0,303519,4151,No Household,0.0,0.0,1.0,...,1.0,1.0,NaN,NaN,524080.611216,527571.964080,1,1,NP = P,NP = P
4,1720010,14.0,SE Snta Clara Cnty,700.0,310217,10534,No Household,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,1,1,No Household,No Housing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34032,2053438,14.0,SE Snta Clara Cnty,709.0,189466,709,No Household,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,155,155,No Household,No Housing
34033,2053472,14.0,SE Snta Clara Cnty,702.0,142028,4220,NP > P,20.0,0.0,0.0,...,18.0,0.0,496724.165197,NaN,NaN,NaN,0,22,NP > P,NP > P
34034,2053524,14.0,SE Snta Clara Cnty,698.0,186270,20436,No Household,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,2,2,No Household,No Housing
34035,2053541,14.0,SE Snta Clara Cnty,702.0,180531,11203,NP < P,0.0,180.0,0.0,...,0.0,149.0,NaN,558068.023906,NaN,NaN,207,0,NP < P,NP < P
